# 기본환경 설정

In [ ]:
# !pip install faiss-cpu

In [ ]:
from google.colab import userdata
HF_KEY = userdata.get("HF_KEY")

In [ ]:
import huggingface_hub
huggingface_hub.login(HF_KEY)

# 모델 로딩

In [ ]:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo langchain-community pypdf langchain_huggingface faiss-cpu
!pip install --no-deps unsloth

In [54]:
from unsloth import FastModel
from langchain.embeddings import HuggingFaceEmbeddings
import torch

In [55]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [56]:
model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 1024*5, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    # device_map = {"": device}
)

==((====))==  Unsloth 2025.8.9: Fast Gemma3 patching. Transformers: 4.55.2. vLLM: 0.10.1.1.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 2. Max memory: 23.494 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [57]:
model = FastModel.for_inference(model)
model = model.eval()

# Custom ChatModel 함수

In [58]:
from typing import Any, Dict, Iterable, Iterator, List, Literal, Optional, Type, Union
from pydantic import BaseModel as PydanticBaseModel
from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.messages import AIMessage, AIMessageChunk, HumanMessage, SystemMessage, BaseMessage, ToolMessage
from langchain_core.outputs import ChatGeneration, ChatResult, ChatGenerationChunk
from langchain_core.tools import BaseTool
from langchain_core.runnables import Runnable, RunnableLambda
from transformers import TextIteratorStreamer
import threading
import re

In [59]:
class GemmaChatModel(BaseChatModel):
    def __init__(self, model, tokenizer, max_tokens: int = 512, do_sample: bool = True, temperature: float = 0.7, top_p: float = 0.9, verbose: bool = False, **kwargs: Any):
        super().__init__()
        object.__setattr__(self, "model", model)
        object.__setattr__(self, "tokenizer", tokenizer)
        object.__setattr__(self, "max_tokens", max_tokens)
        object.__setattr__(self, "do_sample", do_sample)
        object.__setattr__(self, "temperature", temperature)
        object.__setattr__(self, "top_p", top_p)
        object.__setattr__(self, "verbose", verbose)
        object.__setattr__(self, "_gen_lock", threading.Lock())

    ### 공통 유틸 ###########################################################################
    @property
    def _llm_type(self) -> str:
        return "gemma-chat"

    def _format_messages(self, messages: List[BaseMessage]) -> str:
        conv: List[Dict[str, str]] = []
    
        # 1) system 메시지는 처음에만 모아 붙입니다.
        sys_buf = []
        for m in messages:
            if isinstance(m, SystemMessage):
                sys_buf.append(m.content)
        if sys_buf:
            conv.append({"role": "system", "content": "\n".join(sys_buf)})
    
        # 2) 나머지 메시지 변환
        for m in messages:
            if isinstance(m, SystemMessage):
                continue
            elif isinstance(m, HumanMessage):
                conv.append({"role": "user", "content": m.content})
            elif isinstance(m, AIMessage):
                # ⛳️ 중요: 'model' 이 아니라 'assistant'
                conv.append({"role": "assistant", "content": m.content or ""})
            elif isinstance(m, ToolMessage):
                # 템플릿이 'tool' 역할을 모를 수 있으므로 user로 안전 변환
                name = getattr(m, "name", "tool")
                conv.append({"role": "user", "content": f"[{name} 결과]\n{m.content}"})
            else:
                conv.append({"role": "user", "content": str(getattr(m, 'content', m))})
    
        # 3) 역할 번갈음 보장: 동일 역할 연속이면 합치기
        fixed = []
        for m in conv:
            if fixed and fixed[-1]["role"] == m["role"]:
                fixed[-1]["content"] += "\n" + m["content"]
            else:
                fixed.append(m)
    
        # add_generation_prompt 는 마지막이 assistant가 아닐 때만 True
        add_gen = not fixed or fixed[-1]["role"] != "assistant"
    
        # 4) 템플릿 적용, 실패하면 폴백
        try:
            return self.tokenizer.apply_chat_template(
                fixed,
                tokenize=False,
                add_generation_prompt=add_gen,
            )
        except Exception:
            # 아주 단순한 포맷으로 폴백
            s = []
            for m in fixed:
                s.append(f"{m['role'].upper()}: {m['content']}")
            if add_gen:
                s.append("ASSISTANT: ")
            return "\n".join(s)

    def _apply_stop(self, text: str, stop: Optional[List[str]]) -> str:
        if not stop:
            return text
        cut = len(text)
        for s in stop:
            idx = text.find(s)
            if idx != -1:
                cut = min(cut, idx)
        return text[:cut]

    def _build_gen_kwargs(self, **kwargs: Any) -> Dict[str, Any]:
        pad_id = self.tokenizer.pad_token_id
        if pad_id is None:
            pad_id = self.tokenizer.eos_token_id

        eot_id = None
        try:
            eot_id = self.tokenizer.convert_tokens_to_ids("<end_of_turn>")
            if isinstance(eot_id, list):
                eot_id = None
        except Exception:
            eot_id = None

        return {
            "max_new_tokens": kwargs.get("max_tokens", self.max_tokens),
            "do_sample": kwargs.get("do_sample", self.do_sample),
            "temperature": kwargs.get("temperature", self.temperature),
            "top_p": kwargs.get("top_p", self.top_p),
            "eos_token_id": eot_id or self.tokenizer.eos_token_id,
            "pad_token_id": pad_id,
        }

    ### Invoke ############################################################################
    def _generate(self, messages: List[BaseMessage], stop: Optional[List[str]] = None, run_manager: Optional[Any] = None, **kwargs: Any) -> ChatResult:
        prompt = self._format_messages(messages)
        if getattr(self, "verbose", False):
            print("\n[GemmaChatModel/_generate] ==== FINAL PROMPT ====\n" + prompt + "\n=================================================\n")
        # 기본 stop 시퀀스 (ReAct 루프에서 유용)
        if stop is None:
            stop = ["\nObservation:", "\nFinal Answer:"]
        with self._gen_lock:
            inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
            gen_kwargs = self._build_gen_kwargs(**kwargs)
            with torch.no_grad():
                outputs = self.model.generate(**inputs, **gen_kwargs)
        in_len = inputs["input_ids"].shape[1]
        gen_tokens = outputs[0][in_len:]
        decoded = self.tokenizer.decode(gen_tokens, skip_special_tokens=True).strip()
        # 안전망: 종료 마커 제거
        decoded = re.sub(r"(?:</s>|<\|endoftext\|>|<end_of_turn>)+\s*$", "", decoded)
        decoded = self._apply_stop(decoded, stop)
        return ChatResult(generations=[ChatGeneration(message=AIMessage(content=decoded))])

    ### Batch #############################################################################
    def _generate_batch(self, messages_list: List[List[BaseMessage]], stop: Optional[List[str]] = None, run_manager: Optional[Any] = None, **kwargs: Any) -> List[ChatResult]:
        """
        여러 개의 대화를 한 번에 패딩 인코딩하여 generate 가속.
        """
        # [CHANGED] 각 대화를 chat template로 포맷
        prompts = [self._format_messages(msgs) for msgs in messages_list]

        # padding=True, truncation=True 로 배치 인코딩
        tokenized = self.tokenizer(
            prompts, return_tensors="pt", padding=True, truncation=True
        )
        tokenized = {k: v.to(self.model.device) for k, v in tokenized.items()}
        attn = tokenized.get("attention_mask", None)

        gen_kwargs = self._build_gen_kwargs(**kwargs)

        # [CHANGED] 기본 stop 시퀀스
        if stop is None:
            stop = ["</s>", "<end_of_turn>", "<|endoftext|>"]

        with torch.no_grad():
            outputs = self.model.generate(**tokenized, **gen_kwargs)

        results: List[ChatResult] = []
        for i in range(len(prompts)):
            # 각 샘플의 프롬프트 길이만큼 잘라서 신규 토큰만 디코딩
            if attn is not None:
                in_len = int(attn[i].sum().item())
            else:
                in_len = tokenized["input_ids"][i].shape[0]

            gen_tokens = outputs[i][in_len:]
            text = self.tokenizer.decode(gen_tokens, skip_special_tokens=True).strip()
            text = re.sub(r"(?:</s>|<\|endoftext\|>|<end_of_turn>)+\s*$", "", text)  # [CHANGED]
            text = self._apply_stop(text, stop)

            results.append(
                ChatResult(generations=[ChatGeneration(message=AIMessage(content=text))])
            )
        return results

    ### Stream ############################################################################
    def _stream(self, messages: List[BaseMessage], stop: Optional[List[str]] = None, run_manager: Optional[Any] = None, **kwargs: Any) -> Iterator[ChatGenerationChunk]:
        """
        LangChain의 Runnable .stream() 에서 호출되는 내부 스트리밍 제너레이터.
        각 토큰 델타를 ChatGenerationChunk(AIMessageChunk) 로 내보냅니다.
        """
        prompt = self._format_messages(messages)
        if getattr(self, "verbose", False):
            print("\n[GemmaChatModel/_stream] ==== FINAL PROMPT ====")
            print(prompt)
            print("================================================\n")

        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
        gen_kwargs = self._build_gen_kwargs(**kwargs)

        # transformers 스트리머 설정
        streamer = TextIteratorStreamer(
            self.tokenizer, skip_prompt=True, skip_special_tokens=True
        )

        # [CHANGED] 기본 stop 시퀀스
        if stop is None:
            stop = ["</s>", "<end_of_turn>", "<|endoftext|>"]

        # generate를 백그라운드에서 수행
        def _worker():
            with torch.no_grad():
                self.model.generate(**inputs, **gen_kwargs, streamer=streamer)

        th = threading.Thread(target=_worker, daemon=True)
        th.start()

        # 누적 후 stop 시퀀스까지 안전하게 잘라서 내보내기
        buffer = ""
        emitted = 0

        for piece in streamer:
            buffer += piece
            # [CHANGED] 실시간으로 후단 종료 마커 제거(시각적 잔여물 방지)
            tmp = re.sub(r"(?:</s>|<\|endoftext\|>|<end_of_turn>)+\s*$", "", buffer)
            trimmed = self._apply_stop(tmp, stop)

            # 새로 생긴 구간만 델타로 방출
            if len(trimmed) > emitted:
                delta = trimmed[emitted:]
                emitted = len(trimmed)
                yield ChatGenerationChunk(message=AIMessageChunk(content=delta))

            # stop 시퀀스 감지되면 중단
            if stop and len(trimmed) < len(buffer):
                break

        # 스레드 정리(최대한 조용히 종료 대기)
        th.join(timeout=0.1)

    ### Structured output #################################################################
    def _build_json_system_prompt(self, schema_text: str) -> str:
        # 모델이 JSON만 내도록 강하게 지시 (hallucination 방지용 규칙 포함)
        return (
            "You are a strict JSON generator.\n"
            "Return ONLY a single JSON object, no prose, no backticks, no explanations.\n"
            "Do not include trailing commas. Do not include comments.\n"
            "Conform exactly to the following JSON schema (fields, types, required):\n"
            f"{schema_text}\n"
        )

    def _ensure_pydantic(self):
        if PydanticBaseModel is None:
            raise RuntimeError(
                "Pydantic is not available. Install pydantic or pass a dict schema instead of a BaseModel."
            )

    def _schema_to_text(self, schema: Union[Type["PydanticBaseModel"], Dict[str, Any]]) -> str:
        if PydanticBaseModel is not None and isinstance(schema, type) and issubclass(
            schema, PydanticBaseModel
        ):
            try:
                json_schema = schema.model_json_schema()  # pydantic v2
            except Exception:
                json_schema = schema.schema()  # pydantic v1
            return json.dumps(json_schema, ensure_ascii=False, indent=2)
        elif isinstance(schema, dict):
            return json.dumps(schema, ensure_ascii=False, indent=2)
        else:
            raise TypeError(
                "schema must be a Pydantic BaseModel subclass or a dict JSON schema."
            )

    def _parse_structured(self, text: str, schema: Union[Type["PydanticBaseModel"], Dict[str, Any]], include_raw: bool):
        # 코드블록 등 제거 시도(혹시 들어올 경우)
        t = text.strip()
        if t.startswith("```"):
            # ```json ... ``` 또는 ``` ... ```
            t = t.strip("`")
            # 첫 줄에 json 명시가 들어있을 수 있음
            t = "\n".join(
                line for line in t.splitlines() if not line.lower().startswith("json")
            )
        # JSON 파싱
        obj = json.loads(t)

        # Pydantic 검증
        if PydanticBaseModel is not None and isinstance(schema, type) and issubclass(
            schema, PydanticBaseModel
        ):
            validated = (
                schema.model_validate(obj)
                if hasattr(schema, "model_validate")
                else schema.parse_obj(obj)
            )
            return {"parsed": validated, "raw": text} if include_raw else validated
        else:
            # dict 스키마는 별도 검증 없이 반환 (원하면 jsonschema로 검증 가능)
            return {"parsed": obj, "raw": text} if include_raw else obj

    def with_structured_output(self, schema: Union[Type["PydanticBaseModel"], Dict[str, Any]], *, method: Literal["json_mode"] = "json_mode", include_raw: bool = False, system_prefix: Optional[str] = None, deterministic: bool = True):
        schema_text = self._schema_to_text(schema)
        sys_prompt = self._build_json_system_prompt(schema_text)
        if system_prefix:
            sys_prompt = system_prefix.rstrip() + "\n\n" + sys_prompt

        def _invoke(messages_or_any):
            # 입력을 메시지 리스트로 정규화
            if isinstance(messages_or_any, list) and all(
                isinstance(m, BaseMessage) for m in messages_or_any
            ):
                msgs = [SystemMessage(content=sys_prompt)] + messages_or_any
            else:
                # 문자열/딕셔너리 등도 처리
                msgs = [
                    SystemMessage(content=sys_prompt),
                    HumanMessage(content=str(messages_or_any)),
                ]

            # 결정론 옵션
            kw = {}
            if deterministic:
                kw = {"do_sample": False, "temperature": 0.0, "top_p": 1.0}

            # 1차 시도
            result = self._generate(msgs, **kw)
            text = result.generations[0].message.content
            try:
                return self._parse_structured(text, schema, include_raw)
            except Exception:
                # 재시도: 더 강한 지시
                retry_msgs = [
                    SystemMessage(
                        content=sys_prompt + "\nOutput must be valid JSON. Try again."
                    )
                ] + msgs[1:]
                result2 = self._generate(retry_msgs, **kw)
                text2 = result2.generations[0].message.content
                return self._parse_structured(text2, schema, include_raw)

        # Runnable 로 래핑해서 반환 (체인 파이프에 바로 사용 가능)
        return RunnableLambda(_invoke)

    ### Bind Tools ########################################################################
    def _tools_to_specs(self, tools: List[BaseTool]) -> List[Dict[str, Any]]:
        """LangChain Tool -> 간단한 사양(dict)로 변환"""
        specs = []
        for t in tools:
            # 설명
            desc = getattr(t, "description", None) or getattr(t, "description_md", "") or ""
            # 인자 스키마 (가능하면 pydantic schema, 아니면 빈 object)
            schema: Dict[str, Any] = {"type": "object", "properties": {}, "additionalProperties": True}
            args_schema = getattr(t, "args_schema", None)
            if args_schema is not None:
                try:
                    if hasattr(args_schema, "model_json_schema"):
                        schema = args_schema.model_json_schema()  # pydantic v2
                    elif hasattr(args_schema, "schema"):
                        schema = args_schema.schema()             # pydantic v1
                except Exception:
                    pass
            specs.append({
                "name": t.name,
                "description": desc.strip(),
                "parameters": schema
            })
        return specs

    def _build_tool_router_prompt(self, tool_specs: List[Dict[str, Any]]) -> str:
        """도구 선택/호출을 위한 시스템 프롬프트"""
        tools_json = json.dumps(tool_specs, ensure_ascii=False)
        return (
            "You are a tool-using assistant. You may either CALL ONE tool from the list below, "
            "or answer directly. Return ONLY ONE JSON object and NOTHING else.\n\n"
            f"TOOLS (name, description, JSON parameters schema):\n{tools_json}\n\n"
            "OUTPUT FORMAT (choose exactly one):\n"
            "1) Tool call:\n"
            "{ \"tool_name\": \"<one of tool names>\", \"tool_args\": { ... } }\n"
            "2) Final answer (no tool needed):\n"
            "{ \"final\": \"<your answer>\" }\n"
            "Notes:\n"
            "- Do not include prose, code fences, or explanations.\n"
            "- Ensure tool_args strictly follow the tool's JSON schema when possible.\n"
        )

    def bind_tools(
        self,
        tools: List[BaseTool],
        tool_choice: Optional[Union[str, Dict[str, Any]]] = None,
        **kwargs: Any
    ) -> Runnable:
        """
        LangChain 표준: 모델에 툴을 바인딩. 반환 Runnable은 BaseMessage를 돌려야 함.
        - native tool-calling이 없는 로컬 LLM을 프롬프트+파서로 래핑
        """
        tool_specs = self._tools_to_specs(tools)
        sys_prompt = self._build_tool_router_prompt(tool_specs)
        name_set = {t["name"] for t in tool_specs}

        # 고정 tool_choice 지원(선택사항): {"type":"tool","name":"..."} 또는 "tool_name"
        forced_tool: Optional[str] = None
        if isinstance(tool_choice, str):
            forced_tool = tool_choice
        elif isinstance(tool_choice, dict) and tool_choice.get("type") == "tool":
            forced_tool = tool_choice.get("name")

        def _runner(inputs: Union[str, List[BaseMessage], Dict[str, Any]]) -> BaseMessage:
            # 입력 정규화: list[BaseMessage] 또는 {"messages":[...]} 또는 str
            if isinstance(inputs, dict) and "messages" in inputs:
                messages = inputs["messages"]
            elif isinstance(inputs, list) and (len(inputs) == 0 or isinstance(inputs[0], BaseMessage)):
                messages = inputs
            else:
                messages = [HumanMessage(content=str(inputs))]

            # tool_choice가 강제된 경우, 시스템 프롬프트에 힌트 추가
            sys_msg = SystemMessage(content=sys_prompt if forced_tool is None
                                    else sys_prompt + f"\nYou MUST call this tool: {forced_tool}")

            result = self._generate([sys_msg] + messages)  # 기존 _generate 재사용
            text = result.generations[0].message.content or ""

            # 코드펜스 제거 및 JSON 파싱
            t = text.strip()
            if t.startswith("```"):
                t = t.strip("`")
                lines = [ln for ln in t.splitlines() if ln.lower().strip() not in ("json", "jsonc")]
                t = "\n".join(lines)
            try:
                obj = json.loads(t)
            except Exception:
                # JSON 실패 → 도구호출 없이 일반 응답으로 간주
                return AIMessage(content=text)

            # 최종 분기: tool call 또는 final
            if isinstance(obj, dict) and "tool_name" in obj:
                tool_name = str(obj.get("tool_name", "")).strip()
                tool_args = obj.get("tool_args", {}) or {}
                if forced_tool and tool_name != forced_tool:
                    # 강제 도구와 다르게 나오면 강제로 덮어쓰기
                    tool_name = forced_tool
                if tool_name not in name_set:
                    # 존재하지 않는 이름이면 일반 답변 처리
                    return AIMessage(content=text)

                call_id = f"call_{uuid.uuid4().hex[:10]}"
                return AIMessage(
                    content="",
                    tool_calls=[ToolCall(name=tool_name, args=tool_args, id=call_id)]
                )

            if isinstance(obj, dict) and "final" in obj:
                return AIMessage(content=str(obj.get("final", "")))

            # 예상 포맷이 아니면 그냥 텍스트로
            return AIMessage(content=text)

        return RunnableLambda(_runner)

In [60]:
chat_model = GemmaChatModel(model=model, tokenizer=tokenizer, max_tokens=1024*3)

# 기본적인 구성 및 기능 확인 - Runnable: invoke, batch, stream

## invoke

In [61]:
from langchain_core.messages import SystemMessage, HumanMessage

In [62]:
res = chat_model.invoke([
    SystemMessage(content="You are a helpful assistant. Reply in Korean."),
    HumanMessage(content="로컬 Qwen3를 LangChain과 함께 쓰는 법을 요약해줘."),
])

In [63]:
print(res.content)

네, 로컬 Qwen3를 LangChain과 함께 사용하는 방법을 요약해 드리겠습니다. LangChain은 로컬 모델을 쉽게 통합할 수 있도록 도와주는 프레임워크입니다. 다음은 주요 단계와 고려 사항입니다.

**1. 환경 설정:**

*   **Qwen3 모델 다운로드:** Qwen3 모델을 로컬에 다운로드해야 합니다. Hugging Face Hub에서 다운로드하거나, 직접 다운로드할 수 있습니다. (모델 종류 및 크기에 따라 다운로드 시간이 다를 수 있습니다.)
*   **LangChain 설치:** `pip install langchain huggingface_hub` 명령어를 사용하여 LangChain과 Hugging Face Hub를 설치합니다.
*   **필요한 라이브러리 설치:**  `pip install sentence_transformers accelerate` 등 LangChain에서 필요로 하는 라이브러리를 설치합니다.
*   **GPU 설정 (권장):** GPU를 사용하면 추론 속도를 크게 향상시킬 수 있습니다. CUDA가 제대로 설치되어 있는지 확인하고, LangChain에서 GPU를 사용할 수 있도록 설정합니다.

**2. 모델 로드 및 통합:**

*   **Hugging Face Hub 모델 로드:** LangChain의 `HuggingFacePipeline`을 사용하여 Qwen3 모델을 로드합니다. 모델 이름, 모델 경로, 추론 설정(temperature, max_tokens 등)을 지정합니다.
*   **모델 인스턴스 생성:** `HuggingFacePipeline`을 사용하여 모델 인스턴스를 생성합니다.
*   **LangChain 체인 구성:**  Qwen3 모델을 사용하여 텍스트 생성, 질문 답변, 요약 등 다양한 작업을 수행하는 LangChain 체인을 구성합니다.

**3. 코드 예시 (간단한 예시):**

```python
from langchain.llms import HuggingFacePipeline
from la

## batch

In [64]:
from langchain_core.messages import HumanMessage

In [65]:
batch_inputs = [
    [HumanMessage(content="한 문장으로 자기소개해줘.")],
    [HumanMessage(content="파이썬 제너레이터를 간단히 설명해줘.")],
    [HumanMessage(content="서울의 대표 관광지 3곳만 알려줘.")],
]
outs = chat_model.batch(batch_inputs, config={"max_concurrency": 4})

In [66]:
for o in outs:
    print(">>", o.content)

>> 저는 사용자의 질문에 답변하고 다양한 작업을 수행하는 데 도움을 드리는 인공지능 챗봇입니다.
>> 파이썬 제너레이터는 **이터레이터를 생성하는 특별한 함수**입니다. 이터레이터는 필요할 때마다 값을 하나씩 생성하여 반환하는 객체입니다. 제너레이터는 메모리를 효율적으로 사용하면서 무한한 시퀀스를 생성하거나, 데이터 스트림을 처리하는 데 유용합니다.

**핵심 개념:**

* **`yield` 키워드:** 제너레이터 함수는 `yield` 키워드를 사용하여 값을 반환합니다. `yield` 키워드가 호출되면 함수는 현재 상태를 저장하고, 다음 값을 생성하여 반환합니다. 이후 다시 호출되면 저장된 상태에서부터 실행을 재개합니다.
* **이터레이터:** 제너레이터 함수는 이터레이터 객체를 반환합니다. 이터레이터는 `next()` 함수를 사용하여 다음 값을 얻을 수 있습니다.
* **메모리 효율성:** 제너레이터는 모든 값을 한 번에 메모리에 저장하지 않고, 필요할 때마다 값을 생성하기 때문에 메모리 사용량을 줄일 수 있습니다. 특히 무한한 시퀀스를 처리할 때 유용합니다.

**예제:**

```python
def my_generator(n):
  """n개의 숫자를 순서대로 생성하는 제너레이터 함수"""
  for i in range(n):
    yield i

# 제너레이터 객체 생성
gen = my_generator(5)

# 이터레이터 사용
print(next(gen))  # 0 출력
print(next(gen))  # 1 출력
print(next(gen))  # 2 출력

# 제너레이터가 모두 소비되면 StopIteration 예외 발생
# print(next(gen))  # StopIteration 예외 발생
```

**설명:**

1. `my_generator(n)` 함수는 `n`개의 숫자를 순서대로 생성하는 제너레이터 함수입니다.
2. `yield i`는 `i` 값을 반환하고 함수를 일시 중단합니다.
3. `gen = my_generator(5)`는

## stream

In [67]:
from langchain_core.messages import HumanMessage

In [68]:
for chunk in chat_model.stream([HumanMessage(content="Qwen3 모델의 장점과 단점을 알려줘.")]):
    print(chunk.content, end="", flush=True)

Qwen3 모델은 중국의 Alibaba Cloud에서 개발한 대규모 언어 모델(LLM)로, 뛰어난 성능과 효율성을 자랑하며 여러 가지 장점과 단점을 가지고 있습니다.

**장점:**

*   **뛰어난 성능:**
    *   **다양한 벤치마크에서 높은 순위:** Qwen3은 다양한 벤치마크 테스트에서 경쟁 모델들을 능가하는 성능을 보여줍니다. 특히, 중국어 및 영어 기반의 질문 답변, 텍스트 생성, 요약 등 다양한 작업에서 우수한 성능을 보입니다.
    *   **멀티태스크 능력:** Qwen3은 텍스트 생성, 번역, 요약, 질의응답 등 다양한 작업을 동시에 수행할 수 있는 멀티태스크 능력을 갖추고 있습니다.
    *   **긴 문맥 처리 능력:** Qwen3은 긴 문맥을 처리하고 이해하는 능력이 뛰어나, 장문의 텍스트를 기반으로 답변을 생성하거나 요약하는 데 효과적입니다.
*   **효율성:**
    *   **경량화 모델:** Qwen3은 이전 모델에 비해 파라미터 수를 줄이고 효율적인 아키텍처를 사용하여 더 적은 컴퓨팅 자원으로도 높은 성능을 낼 수 있습니다.
    *   **빠른 추론 속도:** Qwen3은 빠른 추론 속도를 제공하여 실시간 응답이 필요한 서비스에 적합합니다.
*   **다양한 버전:** Qwen3은 다양한 크기와 성능을 가진 여러 버전으로 제공되어 사용자의 요구사항에 맞게 선택할 수 있습니다. (예: Qwen3-7B, Qwen3-13B, Qwen3-70B)
*   **오픈 소스:** Qwen3은 오픈 소스로 제공되어 연구 및 개발에 자유롭게 활용할 수 있습니다. (라이선스 확인 필요)
*   **중국어 특화:** 중국어 데이터셋을 기반으로 학습되어 중국어 관련 작업에서 뛰어난 성능을 보입니다.

**단점:**

*   **영어 성능:** Qwen3은 중국어에 비해 영어 성능이 상대적으로 낮을 수 있습니다.
*   **환각 (Hallucination):** LLM의 일반적인 문제점이지만, Qwen3 역시 때때로 사실과 다른 내

## Structured Output

In [69]:
from pydantic import BaseModel, Field

In [70]:
class MovieInfo(BaseModel):
    title: str = Field(..., description="영화 제목")
    year: int = Field(..., description="개봉 연도")
    genres: list[str] = Field(..., description="장르")
    rating: float = Field(..., description="10점 만점 평점")

structured_llm = chat_model.with_structured_output(MovieInfo)

In [71]:
result: MovieInfo = structured_llm.invoke(
    "한국 영화 '괴물'의 제목, 개봉연도, 장르들, 대략적 평점을 JSON으로만 답해줘."
)

In [72]:
print(result)

title='괴물' year=2016 genres=['스릴러', '미스터리', '호러'] rating=8.2


# Agent

## 공통 유틸/도구

In [73]:
from langchain.tools import tool

In [74]:
@tool("multiply")
def multiply_tool(expr: str) -> str:
    """
    두 수의 곱을 반환합니다.
    입력 형식 예: "12 7", "12,7", "12 x 7", "곱하기 12와 7"
    """
    nums = re.findall(r"-?\d+(?:\.\d+)?", expr)
    if len(nums) < 2:
        return "오류: 두 개의 숫자를 찾아야 합니다. 예: '12 7'"
    x, y = float(nums[0]), float(nums[1])
    return str(x * y)

In [75]:
CITY_TO_COUNTRY: Dict[str, str] = {
    "Seoul": "South Korea",
    "Tokyo": "Japan",
    "Paris": "France",
}

In [76]:
@tool("lookup_country")
def lookup_country_tool(city_text: str) -> str:
    """
    도시가 속한 국가를 반환합니다. (모르면 'unknown')
    입력은 도시명만 주거나 문장 속에 포함해도 됩니다. 예: "Paris", "도시는 Seoul"
    """
    # 가장 그럴듯한 '단어' 하나를 도시로 취급
    # (간단히 첫 영문 단어를 우선, 없으면 첫 한글/영문 토큰)
    m = re.search(r"[A-Za-z]+", city_text)
    city = m.group(0) if m else city_text.strip().split()[0]
    return CITY_TO_COUNTRY.get(city, "unknown")

In [77]:
TOOLS = [multiply_tool, lookup_country_tool]

## ReAct : 도구 설명을 보고, 필요한 순간에 툴을 호출하며 추론-행동을 번갈아 수행하는 방식

In [78]:
from langchain.agents import initialize_agent, AgentType

In [79]:
react_agent = initialize_agent(
        tools=TOOLS,
        llm=chat_model,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # 간단한 ReAct 프롬프트 내장
        verbose=True,
    )

In [80]:
output = react_agent.invoke(
    {"input": "12와 7을 곱하고, 'Paris'가 어느 나라 수도인지도 알려줘. 대답은 한국어로 해줘."}
)["output"]



> Entering new AgentExecutor chain...
I need to multiply 12 and 7, and also find out which country Paris is the capital of. I should respond in Korean.
Action: multiply
Action Input: "12 7"
Observation: 84.0
Thought:I now know the result of the multiplication. Next, I need to find out which country Paris is the capital of.
Action: lookup_country
Action Input: "Paris"
Observation: France
Thought:I now know the result of the multiplication and the country Paris is in.
Final Answer: 12와 7을 곱하면 84입니다. 파리는 프랑스 수도입니다.

> Finished chain.


In [81]:
print(output)

12와 7을 곱하면 84입니다. 파리는 프랑스 수도입니다.


## Conversational : 대화 기록을 활용하여 맥락을 유지하는 방식

In [82]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser

In [83]:
history_store = {}

In [84]:
def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in history_store:
        history_store[session_id] = InMemoryChatMessageHistory()
    return history_store[session_id]

In [85]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 유용하고 간결한 한국어 어시스턴트야."),
    MessagesPlaceholder("history"),
    ("human", "{input}"),
])

In [86]:
base_chain = prompt | chat_model | StrOutputParser()

In [87]:
conv_agent = RunnableWithMessageHistory(
    base_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [88]:
session_cfg = {"configurable": {"session_id": "demo-user-1"}}

In [89]:
print("[+] first question")
print(conv_agent.invoke({"input": "Gemma 모델에 대해 설명해 주세요."}, config=session_cfg))
print("\n[+] second question")
print(conv_agent.invoke({"input": "어떤 버전이 있다고 했었나요?"}, config=session_cfg))

[+] first question
Gemma는 구글에서 개발한 **오픈 웨이트 언어 모델(LLM)**입니다. 

**핵심 특징:**

*   **작지만 강력:** Gemma는 다양한 크기의 모델(Gemma 2B, Gemma 7B)로 제공되어, 비교적 적은 컴퓨팅 자원으로도 효율적인 성능을 낼 수 있습니다.
*   **다재다능:** 텍스트 생성, 번역, 질의 응답 등 다양한 자연어 처리 작업에 활용 가능합니다.
*   **오픈 웨이트:** 모델 가중치를 공개하여 누구나 자유롭게 사용, 수정, 배포할 수 있습니다.
*   **안전성 강조:** 유해하거나 편향된 답변을 줄이기 위한 안전 필터링 기술이 적용되어 있습니다.

**주요 정보:**

*   **개발:** 구글
*   **출시:** 2023년 1월
*   **라이선스:** Apache 2.0

**더 자세한 정보는 다음 링크에서 확인하실 수 있습니다:**

*   [Gemma 모델 공식 페이지](https://ai.google.dev/gemma)

궁금한 점이 있다면 언제든지 질문해주세요.

[+] second question
Gemma에는 **Gemma 2B**와 **Gemma 7B** 두 가지 버전이 있습니다.

*   **Gemma 2B:** 더 작고 가벼운 모델로, 비교적 적은 컴퓨팅 자원으로도 빠르게 실행할 수 있습니다.
*   **Gemma 7B:** 더 큰 모델로, 더 복잡한 작업이나 더 높은 성능이 필요한 경우에 적합합니다.

각 모델은 크기와 성능, 그리고 필요한 컴퓨팅 자원 간의 균형을 제공합니다.


## Planning : 복잡한 목표를 받아 계획을 세운 뒤 순차적으로 정답을 찾아가는 방식

In [90]:
# !pip install langchain-experimental

In [91]:
from langchain_experimental.plan_and_execute import (
    load_chat_planner,
    load_agent_executor,
    PlanAndExecute,
)

In [92]:
planner = load_chat_planner(chat_model)
executor = load_agent_executor(chat_model, TOOLS, verbose=True)  # 내부적으로 ReAct 계열
plan_agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

In [93]:
output = plan_agent.invoke(
    {"input": "12와 7을 곱하고, 'Paris'가 어느 나라 수도인지도 알려줘. 대답은 한국어로 해줘."}
)["output"]



> Entering new PlanAndExecute chain...
steps=[Step(value=' Calculate 12 multiplied by 7.'), Step(value=' Determine the capital of France.'), Step(value=' Provide the answer in Korean.'), Step(value=' Calculate 12 * 7 = 84.'), Step(value=' France is a country with Paris as its capital.'), Step(value=" Given the above steps taken, please respond to the users original question.\n\n12와 7을 곱하면 84입니다. 그리고 'Paris'는 프랑스의 수도입니다. (12 * 7 = 84, Paris is the capital of France.) ")]

> Entering new AgentExecutor chain...
Action:```json
{
  "action": "multiply",
  "action_input": "12 7"
}
```
Observation: 84.0
Thought:Action:```json
{
  "action": "Final Answer",
  "action_input": "84"
}
```

> Finished chain.
*****

Step:  Calculate 12 multiplied by 7.

Response: 84

> Entering new AgentExecutor chain...
{
  "action": "lookup_country",
  "action_input": "France"
}

> Finished chain.
*****

Step:  Determine the capital of France.

Response: {
  "action": "lookup_country",
  "action_input": "France"

In [94]:
print(output)

{
  "action": "Final Answer",
  "action_input": "12와 7을 곱하면 84입니다. 그리고 파리는 프랑스의 수도입니다."
}


## Planning : 직접 구현

In [95]:
import json
from typing import List
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.agents import initialize_agent, AgentType

In [96]:
# 1) Planner: 입력 목표를 보고 JSON 배열로 "간단한 단계 목록"을 생성
PLANNER_PROMPT = ChatPromptTemplate.from_messages([
    ("system",
     "너는 계획 수립 보조자야. 사용자의 목표를 달성하기 위한 "
     "간단하고 실행 가능한 단계(step)만 JSON 배열로 출력해. "
     "설명/코드블록 없이 JSON만. 각 원소는 한국어 한 문장으로."),
    ("human",
     "목표: {goal}\n"
     "출력 형식 예시: [\"수치 A와 B의 곱을 계산한다\", \"도시 X의 국가를 조회한다\", \"결과를 한 문장으로 정리한다\"]")
])

In [97]:
# 2) Plan-and-Execute 래퍼
class SimplePlanAndExecute:
    def __init__(self, llm, tools):
        self.llm = llm
        self.planner = PLANNER_PROMPT | chat_model | StrOutputParser()
        self.executor = initialize_agent(tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)  # 스텝 실행은 ReAct로

    def invoke(self, inputs: dict) -> dict:
        goal = inputs["input"] if "input" in inputs else inputs.get("goal", "")
        # (a) 계획 생성
        plan_text = self.planner.invoke({"goal": goal}).strip()
        try:
            steps: List[str] = json.loads(plan_text)
            assert isinstance(steps, list) and all(isinstance(s, str) for s in steps)
        except Exception:
            # 혹시 JSON이 아니면 아주 단순 폴백
            steps = [goal]

        # (b) 스텝별 실행 (각 스텝을 자연어 태스크로 ReAct 에이전트에 던짐)
        observations = []
        for i, step in enumerate(steps, 1):
            out = self.executor.invoke({"input": step})
            obs = out["output"] if isinstance(out, dict) and "output" in out else str(out)
            observations.append(f"[{i}] {step} -> {obs}")

        # (c) 요약/정리 한 번 더 요청(선택)
        summary_prompt = ChatPromptTemplate.from_messages([
            ("system", "다음 관찰 로그를 간결하게 한국어로 종합 보고서 한 문장으로 요약해."),
            ("human", "{log}")
        ])
        summarizer = summary_prompt | self.llm | StrOutputParser()
        summary = summarizer.invoke({"log": "\n".join(observations)})

        return {
            "plan": steps,
            "observations": observations,
            "output": summary.strip(),
        }

In [98]:
planning_agent = SimplePlanAndExecute(chat_model, TOOLS)

In [99]:
res = planning_agent.invoke({
    "input": "1) 15와 4를 곱하고 2) 그 결과와 함께 Seoul의 국가를 한 문장으로 정리"
})



> Entering new AgentExecutor chain...
Action: multiply
Action Input: "15 4"
Observation: 60.0
Thought:Action: lookup_country
Action Input: "Seoul"
Observation: South Korea
Thought:15와 4를 곱한 결과는 60이고, Seoul은 대한민국입니다.
Final Answer: 15와 4를 곱한 결과는 60이고, Seoul은 대한민국입니다.

> Finished chain.


In [100]:
print("\n=== [Planning — no experimental] ===")
print("Plan:", res["plan"])
print("Observations:", *res["observations"], sep="\n")
print("Final:", res["output"])


=== [Planning — no experimental] ===
Plan: ['1) 15와 4를 곱하고 2) 그 결과와 함께 Seoul의 국가를 한 문장으로 정리']
Observations:
[1] 1) 15와 4를 곱하고 2) 그 결과와 함께 Seoul의 국가를 한 문장으로 정리 -> 15와 4를 곱한 결과는 60이고, Seoul은 대한민국입니다.
Final: 15와 4를 곱한 결과는 60이고, 서울은 대한민국입니다.
